In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import warnings, logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

model_nm = 'openai-community/gpt2-large'
model = AutoModelForSequenceClassification.from_pretrained(model_nm, return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(model_nm)

save_path = 'models/gpt2-large'
!mkdir {save_path}
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
!ls {save_path}


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
save_path = 'models/gpt2-large'
model = AutoModelForCausalLM.from_pretrained(save_path, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(save_path)

model_inputs = tokenizer(["The secret to baking a good cake is "], return_tensors="pt").to("mps")

generated_ids = model.generate(**model_inputs, max_length=30)
tokenizer.batch_decode(generated_ids)[0]

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='models/gpt2-large')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

In [ ]:
#This code is for the Llama 3.1 8B Instruct model manually downloaded from Hugging Face.
import transformers
import torch

model_id = "models/llama-3.1-8b-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a French maid, respond and converse in French."},
    {"role": "user", "content": "Comment allez vous?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])